In [5]:
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Read the cities CSV file
cities = pd.read_csv("minard_chard/cities.csv")

# Read the temperature CSV file
temperatures = pd.read_csv("minard_chard/temperature.csv")

# Read the troops CSV file
troops = pd.read_csv("minard_chard/troops.csv")

# Merge the DataFrames together on the long column, using the left join type
df = cities.merge(temperatures, on='long', how='left')
df = df.merge(troops, on='long', how='left')

# Fill in the missing values with NaN
df = df.fillna(method='ffill')

# Create a geopandas dataframe from the city data
city_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat_x']))

# Create a colormap for the temperature column
cmap = plt.cm.hot
norm = mcolors.Normalize(min(df['temp']), max(df['temp']))

# Create a list of colors for each city
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Create a folium map centered on the first city in the DataFrame
m = folium.Map(location=[cities['lat'][0], cities['long'][0]], zoom_start=6, tiles='OpenStreetMap')


# Add markers for each city to the map, colored by temperature
for index, row in city_gdf.iterrows():
    # Create a popup with the city information
    popup_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Temperature: {row['temp']}°C<br>Number of Troops: {row['survivors']}<br>Date: {row['date']}"
    
    # Add a marker for the city to the map
    folium.CircleMarker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)
    
    # Add the city name as a label to the marker
    folium.Marker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)


    # Create a tooltip with the city, temperature, date, and troops information
    tooltip_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Direction: {row['direction']}<br>Temperature: {row['temp']}°C<br>Date: {row['date']}<br>Troops: {row['survivors']}"
    
    # Add a marker for the city to the map
    folium.Marker(location=[row['lat_x'], row['long']], icon=None, popup=row['city'], tooltip=tooltip_text, 
                  color=colors[index % len(colors)], fill_color=colors[index % len(colors)]).add_to(m)

# Display the map
m

In [18]:
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from folium.plugins import FloatImage

# Read the cities CSV file
cities = pd.read_csv("minard_chard/cities.csv")

# Read the temperature CSV file
temperatures = pd.read_csv("minard_chard/temperature.csv")

# Read the troops CSV file
troops = pd.read_csv("minard_chard/troops.csv")

# Merge the DataFrames together on the long column, using the left join type
df = cities.merge(temperatures, on='long', how='left')
df = df.merge(troops, on='long', how='left')

# Fill in the missing values with NaN
df = df.fillna(method='ffill')

# Create a geopandas dataframe from the city data
city_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat_x']))

# Create a colormap for the temperature column
cmap = plt.cm.hot
norm = mcolors.Normalize(min(df['temp']), max(df['temp']))

# Create a list of colors for each city
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Create a folium map centered on the first city in the DataFrame
m = folium.Map(location=[cities['lat'][0], cities['long'][0]], zoom_start=6, tiles='OpenStreetMap')

# Add markers for each city to the map, colored by temperature
for index, row in city_gdf.iterrows():
    # Create a popup with the city information
    # popup_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Temperature: {row['temp']}°C<br>Number of Troops: {row['survivors']}<br>Date: {row['date']}"
    
    # Add a marker for the city to the map
    folium.CircleMarker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)
    
    # Add the city name as a label to the marker
    folium.Marker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)

    # Create a tooltip with the city, temperature, date, and troops information
    tooltip_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Direction: {row['direction']}<br>Temperature: {row['temp']}°C<br>Date: {row['date']}<br>Troops: {row['survivors']}"
    
    # Add a marker for the city to the map
    folium.Marker(location=[row['lat_x'], row['long']], icon=None, popup=row['city'], tooltip=tooltip_text, 
                  color=colors[index % len(colors)], fill_color=colors[index % len(colors)]).add_to(m)

# Create a table of the temperature and dates
temp_table = pd.DataFrame({'Temperature (°C)': df['temp'], 'Date': df['date']})

# Convert the table to an HTML string
temp_table_html = temp_table.to_html(classes='table table-striped table-hover table-condensed table-responsive')

# Create a FloatImage object from the HTML string
temp_table_image = FloatImage(temp_table_html, bottom=0, left=0)

# Add the FloatImage object to the map
temp_table_image.add_to(m)

# Display the map
m

In [48]:
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from folium.plugins import FloatImage
from datetime import datetime

# Read the cities CSV file
cities = pd.read_csv("minard_chard/cities.csv")

# Read the temperature CSV file
temperatures = pd.read_csv("minard_chard/temperature.csv")

# Read the troops CSV file
troops = pd.read_csv("minard_chard/troops.csv")

# Merge the DataFrames together on the long column, using the left join type
df = cities.merge(temperatures, on='long', how='left')
df = df.merge(troops, on='long', how='left')

# Fill in the missing values with NaN
df = df.fillna(method='ffill')

# Create a geopandas dataframe from the city data
city_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat_x']))

# Create a colormap for the temperature column
cmap = plt.cm.jet
norm = mcolors.Normalize(min(df['temp']), max(df['temp']))

# Create a list of colors for each city
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Historical route coordinates based on city coordinates
historical_route = [
    [cities['lat'][0], cities['long'][0]],  # Kowno
    [cities['lat'][1], cities['long'][1]],  # Wilna
    [cities['lat'][2], cities['long'][2]],  # Smorgoni
    [cities['lat'][3], cities['long'][3]],  # Moiodexno
    [cities['lat'][4], cities['long'][4]],  # Gloubokoe
    [cities['lat'][5], cities['long'][5]],  # Minsk
    [cities['lat'][6], cities['long'][6]],  # Studienska
    [cities['lat'][7], cities['long'][7]],  # Polotzk
    [cities['lat'][8], cities['long'][8]],  # Bobr
    [cities['lat'][9], cities['long'][9]],  # Witebsk
    [cities['lat'][10], cities['long'][10]],  # Orscha
    [cities['lat'][11], cities['long'][11]],  # Mohilow
    [cities['lat'][12], cities['long'][12]],  # Smolensk
    [cities['lat'][13], cities['long'][13]],  # Dorogobouge
    [cities['lat'][14], cities['long'][14]],  # Wixma
    [cities['lat'][15], cities['long'][15]],  # Chjat
    [cities['lat'][16], cities['long'][16]],  # Mojaisk
    [cities['lat'][17], cities['long'][17]],  # Moscou
    [cities['lat'][18], cities['long'][18]],  # Tarantino
    [cities['lat'][19], cities['long'][19]],  # Malo-Jarosewii
]

# Create a folium map centered on the first city in the DataFrame
m = folium.Map(location=[historical_route[0][0], historical_route[0][1]], zoom_start=6, tiles='OpenStreetMap')

# Add a line connecting the cities to represent the path of the campaign (in black)
folium.PolyLine(
    locations=historical_route,
    color="black",  # Change the color to black
    opacity=0.7,
    weight=2,
).add_to(m)

# Add markers for each city to the map, colored by temperature
for index, row in city_gdf.iterrows():
    # Create a popup with the city information
    popup_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Temperature: {row['temp']}°C<br>Number of Troops: {row['survivors']}<br>Date: {row['date']}"
    
    # Add a marker for the city to the map
    folium.CircleMarker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)
    
    # Add the city name as a label to the marker
    folium.Marker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)


# Create a table of the temperature and dates
temp_table = pd.DataFrame({'Temperature (°C)': df['temp'], 'Date': df['date']})

# plot the temp+table as a graph

# Convert the table to an HTML string
temp_table_html = temp_table.to_html(classes='table table-striped table-hover table-condensed table-responsive')

# Create a FloatImage object from the HTML string
temp_table_image = FloatImage(temp_table_html, bottom=0, left=0)

# Add the FloatImage object to the map
temp_table_image.add_to(m)

"""
# Plot the temperature data as a graph
plt.figure(figsize=(10, 4))
plt.plot(df['date'], df['temp'], marker='o', linestyle='-', color='b')
plt.title('Temperature Variation Over Time')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.tight_layout()

# Save the graph as an image
plt.savefig('temperature_graph.png')

# Add the temperature graph as an image to the map
folium.Marker(
    location=[historical_route[0][0], historical_route[0][1]],
    icon=folium.CustomIcon(icon_image='temperature_graph.png', icon_size=(300, 150))
).add_to(m)
"""
# Display the map
m
